## TEST - MACHINE LEARNING

_LUCÍA PIQUERAS Y MARTA RUEDAS_

### Objetivo

Partiendo de los datos recogidos en 4 ficheros con información sobre préstamos de diferentes individuos, se procederá a unicarlos todos bajo un dataframe único con el que poder trabajar. Para conseguir el objetivo que nos proponemos, deberemos limpiar todos los datos de los que disponemos, teniendo que seleccionar qué variables son importantes para nuestro objetivo y cuáles desecharemos. Tendremos que realizar toda una serie de pasos hasta quedarnos con los datos que verdaderamente nos sean útiles.

El presente trabajo nace con el objetivo de conseguir un método de predicción eficaz para saber si un cliente de una entidad financiera terminará siendo imapagador de un préstamo concedido, es decir, desarrollar un método que nos diga si debemos o no conceder un préstamo a un individuo que nos lo solicite, como entidad financiera prestamista.

Para un banco es crucial dar pasos sobre seguro y no arriesgarse con individuos que no podrán cumplir con un contrato de préstamo. Por ello, desde hace tiempo incorporan algoritmos y diferentes modelo predictivos para conseguir mantenerse a salvo.

### Librerías

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt 
import seaborn as sns 
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt 
import seaborn as sns 
from math import sqrt 
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split 
from sklearn import model_selection
from sklearn import linear_model
import random 
from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression 
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pickle # guardar modelos

### Importación de datos

In [ ]:
data = pd.read_csv('../data/intermediate/LOAN_var_norm.csv')
data = pd.read_csv('../data/intermediate/LOAN.csv')



# no se si es cargar LOAN o cargar el LOAN normalizado

In [ ]:
# nos lo da el profesor en teoria
data_test = pd.read_csv('./test_set.csv')

In [ ]:
data.index = range(data.shape[0])

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.head(10)

### Variable target: `loan_status`

In [ ]:
data["loan_status"].unique()

In [ ]:
data["loan_status"].value_counts(dropna=False)

- __Eliminamos los NaN's__

In [ ]:
data.dropna(subset=["loan_status"], inplace = True)

- __Eliminamos las observaciones de 'Current' e 'In Grace Period'__ (si hay mas pues mas):

In [ ]:
data = data[data["loan_status"] != "Current"]

In [ ]:
data = data[data["loan_status"] != "In Grace Period"]

In [ ]:
data.shape

In [ ]:
data["loan_status"].value_counts(dropna=False)

Viendo estos resultados, tomamos la decisión de quedarnos __sólo con las observaciones de 'Fully Paid' y 'Charged Off'__.

In [ ]:
data = dat.loc[data["loan_status"].isin(['Fully Paid','Charged Off'])]

In [ ]:
data["loan_status"].value_counts(normalize=False, dropna=False)

In [ ]:
data["loan_status"].value_counts(normalize=True, dropna=False)

Procediendo así, vemos que en el __% de las observaciones si pagaron el préstamo, frente a un % que no__.

Nuestros datos están claramente desbalanceados.

### Tratamiento de las variables

Dado que es imposible trabajar con 151 (o las que sean) variables, hay que establecer distintos filtros que nos ayuden a eliminar aquellas con mucho ruido o que no son útiles. 

Para ello, __eliminaremos en primer lugar, todas aquellas variables que tengan más de un 60% de valores perdidos o NaN's__.

Guardamos en `nans` los % de valores nulos de cada variables:

In [ ]:
nans= data.isnull().mean().sort_values(ascending=False)
nans

Vemos la lista de las variables que tienen más del 60% de valores nulos:

In [ ]:
lista_nulos = sorted(list(nans[nans > 0.60].index))
print(lista_nulos)
len(lista_nulos)

Tenemos un total de __57 (o las que sean) variables con más de un 60% de sus valores nulos__, por tanto, 
pasamos a __eliminar__ dichas variables.

In [ ]:
data.drop(labels=lista_nulos, axis=1, inplace=True)

In [ ]:
data.shape

Nos quedamos por tanto con __94 (o las que sean) variables en el primer filtrado__. 

Las variables elegidas a priori son 24:

- `addr_state`: the state provided by the borrower in the loan application.
- `annual_inc`: the self-reported annual income provided by the borrower during registration.
- `application_type`: indicates whether the loan is an individual application or a joint application with two co-borrowers.
- `dti`: a ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested Lending Club loan, divided by the borrower’s self-reported monthly income.
- `emp_length`: employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
- `emp_title`: the job title supplied by the Borrower when applying for the loan. (Employer Title replaces Employer Name for all loans listed after 9/23/2013).
- `grade`: Lending Club assigned loan grade.
- `home_ownership`: the home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER.
- `initial_list_status`: the initial listing status of the loan. Possible values are – W, F (whole vs. fractional).
- `installment`: the monthly payment owed by the borrower if the loan originates.
- `int_rate`: Interest Rate on the loan.
- `loan_amnt`: the total amount requested by the borrower. The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
- `mort_acc`: number of mortgage accounts.
- `open_acc`: number of open credit lines in the borrower's credit file.
- `pub_rec_bankruptcies`: number of public record bankruptcies (insolvencias de registro público).
- `purpose`: a category provided by the borrower for the loan request.
- `revol_bal`: total credit revolving balance.
- `revol_util`: the amount of credit the borrower is using relative to all available revolving credit. Entendido como la cantidad de crédito que el prestatario pide en relación con todo el crédito rotativo que está a su disposición para pedir.
- `sub_grade` : Lending Club assigned loan subgrade.
- `term`: number of payments on the loan. Values are in months and can be either 36 or 60.
- `title`: the loan title provided by the borrower.
- `total_acc`: the total number of credit lines currently in the borrower's credit file.
- `verification_status`: indicates if income was verified by Lending Club, not verified, or if the income source was verified
- `loan_status`= variable target.

In [ ]:
var_eligidas = ['addr_state', 'annual_inc', 'application_type', 'dti','emp_length', 'emp_title',
                      'grade', 'home_ownership', 'initial_list_status', 'installment',
                      'int_rate', 'loan_amnt', 'mort_acc', 'open_acc',  
                      'pub_rec_bankruptcies', 'purpose', 'revol_bal', 'revol_util', 'sub_grade', 
                      'term', 'title', 'total_acc', 'verification_status','loan_status']

In [ ]:
data = data[var_eligidas]

In [ ]:
data.columns

In [ ]:
data.shape

In [ ]:
data.dtypes

Para los objetos que en realidad sean variables numéricas, por ejemplo porcentajes como las variables int_rate y revol_util:

In [ ]:
# Transformacion a string y operacion de .strip() 
int_rate_strip = data.int_rate.str.strip("%")

In [ ]:
# Convertimos a número:
int_rate_strip = pd.to_numeric(int_rate_strip)

In [ ]:
data["int_rate"] = int_rate_strip

In [ ]:
revol_util_strip = data.revol_util.str.strip("%")

In [ ]:
revol_util_strip = pd.to_numeric(revol_util_strip)

In [ ]:
data["revol_util"] = revol_util_strip

In [ ]:
data.dtypes

## Tratamiento de valores ausentes

In [ ]:
data.isna().sum()

In [ ]:
data_test.isna().sum()

Hay variables que vamos a eliminar:

In [ ]:
data = data.drop(['title','installment','earliest_cr_line','issue_d','sub_grade','addr_state', 'emp_title'],axis=1)

In [ ]:
data_test = data_test.drop(['title','installment','earliest_cr_line','issue_d','sub_grade','addr_state'],axis=1)

In [ ]:
data.isna().sum()

Si variables numéricas presentan valores nulos, imputamos la media:

In [ ]:
data["dti"].fillna(round(data.dti.mean(),2), inplace=True)

In [ ]:
data_test["dti"].fillna(round(data_test.dti.mean(),2), inplace=True)

In [ ]:
data["revol_util"].fillna(round(data.revol_util.mean(),2), inplace=True)



In [ ]:
data_test["revol_util"].fillna(round(data_test.revol_util.mean(),2), inplace=True)



In [ ]:
data_test.isna().sum()

Valores NaN's en variables categóricas:

- Variable __`emp_length`__

Vimos anteriormente como se estructuraba (10+ years, 2 years, < 1 year, 3 years...). Podemos pensar que si no disponemos de esta información sea porque el individuo no ha trabajado nunca o simplemente por un error. Lo que no podemos hacer es eliminar todas las observaciones con NaN's porque perderíamos información. Por tanto:

In [ ]:
data['emp_length'].fillna(0, inplace=True)


In [ ]:
data_test['emp_length'].fillna(0, inplace=True)


(por si hay mas en el conjunto de test que nos de)

Nos quedamos con:

In [ ]:
data.shape

In [ ]:
data_test.shape

Matriz de correlación: 

In [ ]:
variables_numericas = data.select_dtypes(include = [np.number])


In [ ]:
variables_numericas_test = data_test.select_dtypes(include = [np.number])

In [ ]:
corr = variables_numericas.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(600, 1000, as_cmap=True)
ax = sns.heatmap(corr, cmap=cmap, 
xticklabels=corr.columns.values, 
yticklabels=corr.columns.values,
square = True,mask = mask)
plt.gcf().set_size_inches(25,15)
plt.show()

### Data Engineering

Diferenciamos en dos objetos las variables numéricas y las categóricas:

In [ ]:
variables_numericas = data.select_dtypes(include = [np.number])
categoricas = data.select_dtypes(include = [np.object])

In [ ]:
variables_numericas_test = data_test.select_dtypes(include = [np.number])
categoricas_test = data_test.select_dtypes(include = [np.object])


In [ ]:
# Vemos las categóricas ():
variables_categoricas.columns

In [ ]:
# Vemos las numéricas ():
variables_numericas.columns

In [ ]:
# Vemos las numéricas ():
variables_numericas_test.columns

In [ ]:
# Vemos las categóricas ():
variables_categoricas_test.columns

### Normalizamos las variables numéricas

In [ ]:
scaler = preprocessing.StandardScaler()

numericas_escaladas = scaler.fit_transform(variables_numericas)
numericas_escaladas = pd.DataFrame(numericas_escaladas, columns = [variables_numericas])

In [ ]:
scaler = preprocessing.StandardScaler()

numericas_escaladas_test = scaler.fit_transform(variables_numericas_test)
numericas_escaladas_test = pd.DataFrame(numericas_escaladas_test, columns = [variables_numericas_test])

In [ ]:
numericas_escaladas.head()

In [ ]:
numericas_escaladas_test.head()

## Dummies categóricas

### 1. Hacemos dummie la variable `loan_status`

In [ ]:
variables_categoricas['Charged_off'] = (variables_categoricas['loan_status'] == 'Charged Off').apply(np.uint8)
variables_categoricas.drop('loan_status', axis=1, inplace=True)


In [ ]:
variables_categoricas_test['Charged_off'] = (variables_categoricas_test['loan_status'] == 'Charged Off').apply(np.uint8)
variables_categoricas_test.drop('loan_status', axis=1, inplace=True)


### 2. Hacemos dummies en el resto

Los algoritmos de aprendizaje automático requieren que las variables de entrada y salida sean numéricas. Esto implica que los datos categóricos deben ser codificados en números antes de que podamos usarlos para ajustar y evaluar un modelo. Hay muchas maneras de codificar variables categóricas para el modelado.

Utilizamos el método de _One-Hot Encoding_ con la función `pd.get_dummies`.

In [ ]:
# todas menos charged off
variables_categoricas = pd.get_dummies(variables_categoricas, columns=['application_type', 'home_ownership', 'initial_list_status', 'purpose',
       'sub_grade', 'term', 'verification_status'])




In [ ]:
# todas menos charged off
variables_categoricas_test = pd.get_dummies(variables_categoricas, columns=['application_type', 'home_ownership', 'initial_list_status', 'purpose',
       'sub_grade', 'term', 'verification_status'])




In [ ]:
variables_categoricas

In [ ]:
variables_categoricas_test

In [ ]:
variables_categoricas.index = range(variables_categoricas.shape[0])

In [ ]:
variables_categoricas_test.index = range(variables_categoricas_test.shape[0])



In [ ]:
variables_categoricas.dtypes

In [ ]:
variables_categoricas_test.dtypes

Concatenamos los dataframes:

In [ ]:
data_tratada = pd.concat([numericas_escaladas, variables_categoricas], axis = 1)

In [ ]:
data_tratada_test = pd.concat([numericas_escaladas_test, variables_categoricas_test], axis = 1)



In [ ]:
data_tratada

In [ ]:
data_tratada_test

Procedemos a lanzar el modelo que mejores resultados nos ha dado en el entrenamiento, a pesar de que todos nos han dado más o menos el mismo _accuracy_ .

Este modelo ha sido el SVM, tras hacer el grid y elegir los hiperparámetros óptimos. 

## Train y Test

Para reducir el coste computacional, hemos ido probando a tirar los modelos con no todos los datos para intentar encontrar un punto a partir del cual los modelos no nos mejoren aún aumentando la cantidad de observaciones.

Creemos que a más datos mejor, pero que los algoritmos quizás no necesitan tantos datos para obtener buenos resultados, y tirándolos con los más de 400k datos, se nos eternizaba tanto que nunca terminaba de salir, por ejemplo los grids.

Lanzaremos con 100k.

In [ ]:
data_train = data.sample(n=100000, random_state=1234)

In [ ]:
Y = data_tratada['Charged_off']
X = data_tratada.drop(['Charged_off'], axis = 1)


¿hay que hacer test?

In [ ]:
Y_test = data_tratada_test['Charged_off']
X_test = data_tratada_test.drop(['Charged_off'], axis = 1)

In [ ]:
undersample = RandomUnderSampler(sampling_strategy = 'majority')

In [ ]:
x_under, y_under = undersample.fit_sample(X, Y)





## Modelo SVM 

In [2]:
SVM = SVC(kernel='rbf',
          tol=0.01, 
          C=1, random_state=1234)

In [ ]:
SVM.fit(x_under, y_under)

In [ ]:
SVM.score(x_under, y_under)

In [ ]:
SVM_predicted = SVM.predict(x_test)

In [ ]:
SVM.score(x_test, y_test)

In [ ]:
print(confusion_matrix(y_test, SVM_predicted))

In [ ]:
print(classification_report(y_test, SVM_predicted))

In [ ]:
metrics.plot_roc_curve(SVM, x_test, y_test)
plt.show()

### Búsqueda Grid

In [ ]:
# probamos con kernel radial solo y ya nos tarda mucho
# dejamos el gamma por defecto (no lo ponemos)
# metemos 5 en el parametro de regularizacion
# njobs = 3 para usar 3 procesadores
 
param_grid = {'C': [0.1, 10, 100],
              'gamma' : [0.001, 0.0001],
              'kernel': ['rbf']}  
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3, n_jobs = 3)
grid.fit(x_under, y_under)

In [ ]:
# para ver cuál es el mejor modelo
print(grid.best_estimator_) 
# SVC(C=10, gamma=0.001)

In [ ]:
# guardamos el mejor modelo
SVM_Best = grid.best_estimator_

In [ ]:
grid_predictions = SVM_Best.predict(x_test)
print(classification_report(y_test, grid_predictions))

In [ ]:
metrics.plot_roc_curve(SVM_Best, x_test, y_test)
plt.show()